Generating precipitation data for landslide or non landslide points based on date and latitude/longitude.

Using Earth Engine Maps of precipitation data in last 7 days

In [3]:
# We import earth engine, authenticate and initialize
import ee
import geemap
# ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()


In [6]:
PATH_INPUT_GEOJSON = './data_train.geojson'
PATH_OUTPUT_GEOJSON = './data_with_precipitation.geojson'
PATH_OUTPUT_CSV = './data_with_precipitation.csv'

In [7]:
# The usefull dataset of maps of precipitation collections
gsmap_precipitation_collection_2014 = ee.ImageCollection("JAXA/GPM_L3/GSMaP/v6/operational") #https://developers.google.com/earth-engine/datasets/catalog/JAXA_GPM_L3_GSMaP_v6_operational
gsmap_precipitation_collection_2000 = ee.ImageCollection("JAXA/GPM_L3/GSMaP/v6/reanalysis") #https://developers.google.com/earth-engine/datasets/catalog/JAXA_GPM_L3_GSMaP_v6_reanalysis
global_precipitation_measurement = ee.ImageCollection("NASA/GPM_L3/IMERG_V06") # https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06#bands


In [9]:
import datetime
# Dates for each of the datasets available
gsmap_precipitation_collection_2014_range = [datetime.datetime.strptime('2014-03-01','%Y-%m-%d'), datetime.datetime.strptime('2022-10-11','%Y-%m-%d')]
gsmap_precipitation_collection_2000_range = [datetime.datetime.strptime('2000-03-01','%Y-%m-%d'), datetime.datetime.strptime('2014-03-12','%Y-%m-%d')]
global_precipitation_measurement_range = [datetime.datetime.strptime('2000-06-01','%Y-%m-%d'), datetime.datetime.strptime('2022-10-05','%Y-%m-%d')]

In [11]:
# Load geojson with training data to generate
import geojson
from datetime import datetime,timedelta
geojson_file = open(PATH_INPUT_GEOJSON, 'r', encoding="utf8")
geojson = geojson.load(geojson_file)


In [24]:
len(geojson.features)

10190

In [14]:
# Function to get landslide precipitation data of geojson points and append them to properties
def get_landslides_precipitation_data(geojson):
    error_landslides_ids_info_precipitation = []
    landslides_not_includes_in_info_precipitation = []
    count = 0
    for feature in geojson.features:
        count += 1
        geometry = feature.geometry
        coordinates = geometry.coordinates
        properties = feature.properties
        new_precipitacion_properties = {}
        landslide_id = properties["landslide_id"]
        event_date = properties["event_date"]
        landslide_prefix =  "" if properties["landslide"]=="1" else "non_"
        region = ee.Geometry.Point(coords=coordinates)
        # Search last 7 days of precipitation 
        end_date = datetime.strptime(event_date,'%Y-%m-%d')
        init_date = datetime.strptime(event_date,'%Y-%m-%d') - timedelta(days=7)
        try:
            #gsmap_precipitation_collection_2014
            if(init_date>gsmap_precipitation_collection_2014_range[0]):
                collection_region = gsmap_precipitation_collection_2014.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region)
                hourly_precip_rate_sum_dataset = collection_region.select('hourlyPrecipRate').sum()
                pixel_info_hourly_precip_rate_sum = hourly_precip_rate_sum_dataset.reduceRegion(
                    geometry=region,
                    scale=10,
                    reducer=ee.Reducer.mean() 
                )
                

                new_precipitacion_properties = {
                    'hourly_precip_rate_sum': pixel_info_hourly_precip_rate_sum.get('hourlyPrecipRate').getInfo(),
                }
            #gsmap_precipitation_collection_2000
            elif(init_date>gsmap_precipitation_collection_2000_range[0]):
                collection_region = gsmap_precipitation_collection_2000.filterDate(str(init_date).split(" ")[0], str(end_date).split(" ")[0]).filterBounds(region)
                hourly_precip_rate_sum_dataset = collection_region.select('hourlyPrecipRate').sum()
                pixel_info_hourly_precip_rate_sum = hourly_precip_rate_sum_dataset.reduceRegion(
                    geometry=region,
                    scale=10,
                    reducer=ee.Reducer.mean() 
                )
                
                new_precipitacion_properties = {
                    'hourly_precip_rate_sum': pixel_info_hourly_precip_rate_sum.get('hourlyPrecipRate').getInfo(),
                    # 'hourly_precip_rate_mean':pixel_info_hourly_precip_rate_mean.get('hourlyPrecipRate').getInfo()
                }
                
            else:
                    landslides_not_includes_in_info_precipitation.append(landslide_id)
            
            feature.properties = dict(properties.items() | new_precipitacion_properties.items())
            print("{}/{} completed".format(count,len(geojson.features)))
        except Exception as e: 
            print(e)
            error_landslides_ids_info_precipitation.append(landslide_id)
            landslides_not_includes_in_info_precipitation.append(landslide_id)
            pass
    return geojson, error_landslides_ids_info_precipitation, landslides_not_includes_in_info_precipitation

In [15]:
results = get_landslides_precipitation_data(geojson=geojson)

1/10190 completed
2/10190 completed
3/10190 completed
4/10190 completed
5/10190 completed
6/10190 completed
7/10190 completed
8/10190 completed
9/10190 completed
10/10190 completed
11/10190 completed
12/10190 completed
13/10190 completed
14/10190 completed
15/10190 completed
16/10190 completed
17/10190 completed
18/10190 completed
19/10190 completed
20/10190 completed
21/10190 completed
22/10190 completed
23/10190 completed
24/10190 completed
25/10190 completed
26/10190 completed
27/10190 completed
28/10190 completed
29/10190 completed
30/10190 completed
31/10190 completed
32/10190 completed
33/10190 completed
34/10190 completed
35/10190 completed
36/10190 completed
37/10190 completed
38/10190 completed
39/10190 completed
40/10190 completed
41/10190 completed
42/10190 completed
43/10190 completed
44/10190 completed
45/10190 completed
46/10190 completed
47/10190 completed
48/10190 completed
49/10190 completed
50/10190 completed
51/10190 completed
52/10190 completed
53/10190 completed
54

In [25]:
geojson_result = results[0]

In [27]:
ids = []
duplicates = []
for feature in geojson_results.features:
    properties = feature.properties
    landslide_id = properties['landslide_id']
    if landslide_id in ids:
        duplicates.append(landslide_id)
    ids.append(landslide_id)
print(len(duplicates))

0


In [29]:
import csv
with open(PATH_OUTPUT_CSV, 'w') as file:
    writer = csv.writer(file)
    # 3. step
    data = ["landslide_id", "precipitation_mm_last_7_days"]
    writer.writerow(data)
    index = 0
    for feature in geojson_results.features:
        index +=1
        properties = feature.properties
        data = [properties['landslide_id'],properties['hourly_precip_rate_sum']]
        writer.writerow(data)
    file.close()